## Binary Classification ##
 - Implement by Piston Yang

In [1]:
import numpy as np

In [2]:
sample = np.array([[0, 1], [4, -1], [2, 1], [3, -1], [1, 1],
                  [5, -1], [6, 1], [7, 1], [8, 1], [9, -1]])
sample

array([[ 0,  1],
       [ 4, -1],
       [ 2,  1],
       [ 3, -1],
       [ 1,  1],
       [ 5, -1],
       [ 6,  1],
       [ 7,  1],
       [ 8,  1],
       [ 9, -1]])

In [3]:
def init_first_step_weight(num_sample):
#     初始化数据权值分布
    return np.array([1/num_sample for i in range(num_sample)])

In [4]:
def sort_sample(sample):
#     对输入数据进行排序,方便算法计算
# exp:IN:array([
#        [ 0,  1],
#        [ 4,  1],
#        [ 2,  1],
#        [ 3, -1],
#        [ 1, -1]])
#     OUT:array([
#        [ 0,  1],
#        [ 1, -1],
#        [ 2,  1],
#        [ 3, -1],
#        [ 4,  1]])
    return sample[sample[:,0].argsort()]

### 寻找使误差率error最小的分类点 ###
- 这个函数暂时仅以能实现为目的编写，不考虑任何优化
- 此实现不能以0.5为阈值提前结束搜索，否则可能无法查找到最优error
- 时间复杂度为O(2n)
- 此函数为了快速实现对数据集敏感x必须为np.arange(0, N, 1), y~{1， -1}.

In [5]:
def serch_minimize_error_point(D, sample):
#     D[i]对应sample[i]的权重
    sap_len = len(sample)


#     先正向搜索：即 G(x): 1: x<v
#                         -1: x>v
#     实际上有n+1个分裂点
    f_font_err = 0.
    f_behind_error = np.sum(D[sample[:,1] != -1])
    forward_err = f_font_err + f_behind_error
    
    f_split_point = 0.5  #最左边
#     每遍历过一个元素根据其类别更新f_font_err和f_behind_err如果error变小了则更新forward_err
    for i in range(sap_len):
        split_point = i + 0.5
#         如果当前y值==1：则f_font_err不变，f_behind_error减小
        if sample[i][1] == 1:
            f_behind_error -= D[i]
            if f_font_err + f_behind_error < forward_err:
#                 如果当前error最小则更新forward_err和最优分裂点
                forward_err = f_font_err + f_behind_error
                f_split_point = split_point

#         如果当前y值==-1：则f_font_err增大，f_behind_error不变
        else:
            f_font_err += D[i]
#             不能提前退出，会出现问题，跟我的实现方式有关
#             if f_font_err + f_behind_error > 0.5:
#                 break



#     反向搜索：即 G(x):  1: x>v
#                        -1: x<v
#     不会改变for循环的方向
    b_font_err = 0.
    b_behind_err = np.sum(D[sample[:,1] != 1])
    backward_err = b_font_err + b_behind_err
    b_split_point = 0.5

    for i in range(sap_len):
        split_point = i + 0.5
        
        if sample[i][1] == 1:
            b_font_err += D[i]
#             if b_font_err + b_behind_err > 0.5:
#                 break
        else:
            b_behind_err -= D[i]
            if b_font_err + b_behind_err < backward_err:
                backward_err = b_font_err + b_behind_err
                b_split_point = split_point

#     判断哪种方式得到的error最小并确定搜索方式

    if forward_err <= backward_err:
        min_error = forward_err
        final_point = f_split_point
        serch_type = 'forward'
    else:
        min_error = backward_err
        final_point = b_split_point
        serch_type = 'backward'
    Gm = {'sp': final_point, 'type': serch_type}
    return min_error, Gm

### 根据error计算当前分类器权重alpha ###

In [6]:
# alpha m
def compute_alpha(error):
    return 1/2 * np.log((1-error)/error)

### 更新训练数据的权值分布D ###

In [7]:
# Gm即G(x)为一个dict{'sp' : split_point, 'type': serch_type}
def update_weight(sample, Dm, alpha, Gm):
    N = len(sample)
    split_point = Gm['sp']
    serch_type = Gm['type']

    if serch_type == 'forward':

        Gm_ems = np.array([1. if i < split_point else -1. for i in range(N)])
    else:

        Gm_ems = np.array([-1. if i < split_point else 1. for i in range(N)])

    yi = sample[:, 1]

    Zm = np.sum(Dm * np.exp(-alpha * yi * Gm_ems))

    D_update = Dm / Zm * np.exp(-alpha * yi * Gm_ems)
    
    return D_update

### 根据当前已训练好的分类器，在训练集上统计预测错的数据总数 ###
- 此函数会帮助判断算法是否已经可以结束
- 并且可以记录每轮算法分类错的点

In [8]:
# fx为一个list，list中的元素为dict，dict中保存每一个fi(x)的alpha和G(x)
def get_misclassification_point(sample, fx):
#     计算误分类点，fx为组合分类器，要先计算组合分类器给出的0/1概率总和，然后根据最大值决定分类器输出
#     再比较true_label判断是否为误分类点
    num_Gm = len(fx)
    len_smp = len(sample)
    misclassification_point = []
#     获取组合分类器对y_hat_n的结果
    for n in range(len_smp):
#         统计组合分类器对正负样本的概率
        pos_prop = 0.
        neg_prop = 0.
        for m in range(num_Gm):
            Gm = fx[m]['Gm']
            alpha = fx[m]['alpha']
            Gm_p = Gm['sp']
            Gm_type = Gm['type']
            
            if Gm_type == 'forward':
                if n < Gm_p:  #正样本
                    pos_prop += alpha
                else:  #负样本
                    neg_prop += alpha
            else:
                if n < Gm_p:  #负样本
                    neg_prop += alpha
                else:  #正样本
                    pos_prop += alpha
        if pos_prop > neg_prop:
            y_hat_n = 1
        else:
            y_hat_n = -1
        y_n = sample[n][1]
        
        if y_n != y_hat_n:
#             print("true lab: %d, Pred_lab: %d." % (y_n, y_hat_n))
            misclassification_point.append(n)
    return misclassification_point

### 算法真正迭代 ###
- 测试数据来自《统计学习方法》——李航 P140-P142
- 无差别复现例题

In [9]:
# max_N定义最大迭代次数
def AdaBoost(max_N, sample):
    sample = sort_sample(sample)
    len_sap = len(sample)
    
    cache_D = []   #迭代过程中数据权值分布
    cache_misclassification_point = []  #每次迭代组合分类器分类错误的点
    cache_min_error = []   #每次迭代的min_error——很重要，一般就这个函数出错
    fx = []    #组合分类器
    num_iter = 0   #迭代次数
    
    D = None  #每轮迭代的权值分布，提前定义，以免循环中途被释放
    for m in range(1, max_N+1):
        if m == 1:  #如果是第一次迭代初始化权值分布
            D = init_first_step_weight(len_sap)
            cache_D.append(D)
#         使用具有权值分布Dm的训练数据集学习，得到基本分类器Gm,同时计算Gm在训练数据集上的分类误差率（其实是根据最小的误差率决定Gm）
        min_error, Gm = serch_minimize_error_point(D, sample)
        cache_min_error.append(min_error)
#         计算Gm的系数
        alpha = compute_alpha(min_error)
#         更新训练数据集的权值分布
        D = update_weight(sample, D, alpha, Gm)
        cache_D.append(D)
#         构建基本分类器
        fxi = {'alpha': alpha, 'Gm':Gm}
#         组合基本分类器
        fx.append(fxi)
#         判断组合分类器是否能完全正确分类样本/训练集
        misclassification_point = get_misclassification_point(sample, fx)
        cache_misclassification_point.append(misclassification_point)
#         如果组合分类器可以完全正确分类样本，则提前结束迭代
        if len(misclassification_point) == 0:
            num_iter = m
            break
        if m == max_N:
            num_iter = m
#     返回所有迭代过程中的数据，可以帮助排查算法错误，按需获取
    return cache_D, cache_misclassification_point, cache_min_error, fx, num_iter  

In [10]:
cache_D, cache_misclassification_point, cache_min_error, fx_final, num_iter = AdaBoost(10, sample)

In [11]:
cache_D

[array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1]),
 array([ 0.07142857,  0.07142857,  0.07142857,  0.07142857,  0.07142857,
         0.07142857,  0.16666667,  0.16666667,  0.16666667,  0.07142857]),
 array([ 0.04545455,  0.04545455,  0.04545455,  0.16666667,  0.16666667,
         0.16666667,  0.10606061,  0.10606061,  0.10606061,  0.04545455]),
 array([ 0.125     ,  0.125     ,  0.125     ,  0.10185185,  0.10185185,
         0.10185185,  0.06481481,  0.06481481,  0.06481481,  0.125     ])]

In [12]:
cache_misclassification_point

[[6, 7, 8], [3, 4, 5], []]

In [13]:
cache_min_error

[0.30000000000000004, 0.21428571428571438, 0.18181818181818196]

In [14]:
fx_final

[{'Gm': {'sp': 2.5, 'type': 'forward'}, 'alpha': 0.42364893019360172},
 {'Gm': {'sp': 8.5, 'type': 'forward'}, 'alpha': 0.64964149206513011},
 {'Gm': {'sp': 5.5, 'type': 'backward'}, 'alpha': 0.75203869838813653}]

In [15]:
num_iter

3